<a href="https://colab.research.google.com/github/Jasur870/Scalping_bot/blob/main/%D0%94%D0%BE%D0%B1%D1%80%D0%BE_%D0%BF%D0%BE%D0%B6%D0%B0%D0%BB%D0%BE%D0%B2%D0%B0%D1%82%D1%8C_%D0%B2_Colab!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install python-telegram-bot ta nest_asyncio matplotlib requests pandas

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.3/702.3 kB 13.8 MB/s eta 0:00:00
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=17ddf6d0d241e81455166c947a5cdc400769b6444457f3cc3daad3c522644196
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [5]:

import pandas as pd
import matplotlib.pyplot as plt
import requests
import io
from ta.trend import EMAIndicator, MACD
from ta.volatility import BollingerBands
from telegram import Bot
from datetime import datetime
import nest_asyncio
import asyncio

# ==== API va sozlamalar ====
TELEGRAM_TOKEN = "8073094606:AAFC6tSJWVhhkfNbifYXVcZUgwZJ82QDU1o"
CHAT_ID = "636914093"
API_KEY = "1c10752e92ea481ca2992336e5700c74"
SYMBOLS = ["BTC/USD", "ETH/USD", "BNB/USD", "SOL/USD", "ADA/USD"]
INTERVAL = "15min"
LOOKBACK = 50

bot = Bot(token=TELEGRAM_TOKEN)

# ==== Chart pattern aniqlash funksiyalari ====
def is_head_and_shoulders(df):
    closes = df['close'].values[-7:]
    if len(closes) < 7:
        return False
    left_shoulder = closes[0] < closes[1] > closes[2]
    head = closes[2] < closes[3] > closes[4]
    right_shoulder = closes[4] < closes[5] > closes[6]
    return left_shoulder and head and right_shoulder

def is_double_top(df):
    closes = df['close'].values[-5:]
    if len(closes) < 5:
        return False
    return closes[0] < closes[1] > closes[2] < closes[3] > closes[4]

def is_double_bottom(df):
    closes = df['close'].values[-5:]
    if len(closes) < 5:
        return False
    return closes[0] > closes[1] < closes[2] > closes[3] < closes[4]

# Shu yerga yana 17 ta pattern funksiyasini qo'shishingiz mumkin

def detect_patterns(df):
    patterns_found = []
    if is_head_and_shoulders(df):
        patterns_found.append("Head and Shoulders")
    if is_double_top(df):
        patterns_found.append("Double Top")
    if is_double_bottom(df):
        patterns_found.append("Double Bottom")
    # Boshqa patternlarni ham qo'shish mumkin...
    return patterns_found if patterns_found else ["No clear pattern"]

# ==== Ma'lumot olish ====
def fetch_data(symbol):
    url = f"https://api.twelvedata.com/time_series?symbol={symbol}&interval={INTERVAL}&outputsize={LOOKBACK}&apikey={API_KEY}&format=JSON"
    r = requests.get(url).json()
    if 'values' not in r:
        raise ValueError(f"❌ Xatolik {symbol} uchun: {r.get('message', 'No values')}")
    df = pd.DataFrame(r['values'])
    df['datetime'] = pd.to_datetime(df['datetime'])
    df.set_index('datetime', inplace=True)
    df = df.sort_index()
    df = df.astype(float)
    return df

# ==== Texnik indikatorlar va analiz ====
def analyze(df):
    ema = EMAIndicator(close=df['close'], window=20).ema_indicator()
    macd = MACD(close=df['close']).macd_diff()
    boll = BollingerBands(close=df['close'], window=20)
    bb_high = boll.bollinger_hband()
    bb_low = boll.bollinger_lband()

    df['ema'] = ema
    df['macd'] = macd
    df['bb_high'] = bb_high
    df['bb_low'] = bb_low

    signal = "Hold"
    if df['close'].iloc[-1] > df['ema'].iloc[-1] and df['macd'].iloc[-1] > 0:
        signal = "Buy"
    elif df['close'].iloc[-1] < df['ema'].iloc[-1] and df['macd'].iloc[-1] < 0:
        signal = "Sell"

    return df, signal

# ==== Grafik yaratish ====
def plot_chart(df, symbol, signal, patterns):
    fig, ax = plt.subplots(figsize=(12, 6))
    df[['close', 'ema']].plot(ax=ax, lw=2)
    ax.fill_between(df.index, df['bb_high'], df['bb_low'], color='lightgray', alpha=0.3)
    ax.set_title(f"{symbol} - Signal: {signal}")
    # Patternlarni grafikga yozish
    pattern_text = "\n".join(patterns)
    plt.text(df.index[int(len(df)/2)], max(df['close']), f"Patterns:\n{pattern_text}", fontsize=12, color='red')
    ax.grid(True)
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    plt.close()
    return buf

# ==== Telegramga yuborish ====
async def send_signal(symbol):
    try:
        df = fetch_data(symbol)
        df, signal = analyze(df)
        patterns = detect_patterns(df)

        # Patternlarni signalga qo'shish, agar patternlar mavjud bo'lsa
        pattern_text_for_signal = ", ".join([p for p in patterns if p != "No clear pattern"])

        # Signal sababi:
        reason = ""
        if signal == "Buy":
            reason = "Close narxi EMA dan yuqori va MACD ijobiy"
        elif signal == "Sell":
            reason = "Close narxi EMA dan past va MACD manfiy"

        if pattern_text_for_signal:
            reason += f", Pattern: {pattern_text_for_signal}"

        signal_with_reason = f"{signal} ({reason})" if reason else signal

        img = plot_chart(df, symbol, signal_with_reason, patterns)
        caption = (
            f"*{symbol}*\n"
            f"📊 Signal: {signal_with_reason}\n"
            f"📈 Patterns: {', '.join(patterns)}\n"
            f"🕒 {datetime.utcnow().strftime('%Y-%m-%d %H:%M UTC')}"
        )
        await bot.send_photo(chat_id=CHAT_ID, photo=img, caption=caption, parse_mode='Markdown')
        await asyncio.sleep(1.5)  # Telegram spamiga yo'l qo'ymaslik uchun
    except Exception as e:
        await bot.send_message(chat_id=CHAT_ID, text=f"❌ Xatolik {symbol} uchun: {str(e)}")
        await asyncio.sleep(1.5)

# ==== Asosiy ishga tushirish funksiyasi ====
async def main():
    tasks = [send_signal(symbol) for symbol in SYMBOLS]
    await asyncio.gather(*tasks)

# ==== Google Colab uchun asyncio tuzatish ====
import nest_asyncio
nest_asyncio.apply()
await main()